In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [151]:
import warnings
warnings.filterwarnings("ignore")

In [152]:
WC=pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
WC.drop(['Unnamed: 0'],axis=1,inplace=True)

In [153]:
WC=WC.drop_duplicates()

In [154]:
WC.dropna(subset=['Review Text'],inplace=True)

In [155]:
WC['Recommended IND']=np.where(WC['Recommended IND']==1,0,1)  # 0--recommendation, 1-- not recommendation

In [156]:
WC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22640 entries, 0 to 23485
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Clothing ID              22640 non-null  int64 
 1   Age                      22640 non-null  int64 
 2   Title                    19675 non-null  object
 3   Review Text              22640 non-null  object
 4   Rating                   22640 non-null  int64 
 5   Recommended IND          22640 non-null  int32 
 6   Positive Feedback Count  22640 non-null  int64 
 7   Division Name            22627 non-null  object
 8   Department Name          22627 non-null  object
 9   Class Name               22627 non-null  object
dtypes: int32(1), int64(4), object(5)
memory usage: 1.8+ MB


In [157]:
WC['Recommended IND'].value_counts()

0    18539
1     4101
Name: Recommended IND, dtype: int64

# clean  and preprocess reveiw text

In [158]:
def prior_split(data):
    data=re.sub("n't",'not',data)
    data=re.sub("i'm",'i am',data)
    data=re.sub("I'm","I am",data)
    data=re.sub("it's","it is",data)
    data=re.sub("It's","It is",data)
    return data
WC['Review Text']=WC['Review Text'].apply(prior_split)

In [159]:
# decided the split method 
def split_method(data):
    split_word=re.split('\W+',data)
    return split_word

In [160]:
# the stop words 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
stop_words=stopwords.words("english")
new_stopping_words = stop_words[:len(stop_words)-36]
new_stopping_words.remove("not") 
new_stopping_words.extend(['would','could',])
new_stopping_words.remove("but")
type_class=[word.lower() for word in list(WC['Class Name'].unique())[:-3]]
new_stopping_words.extend(type_class)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zheng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [195]:
type_class

['intimates',
 'dresses',
 'pants',
 'blouses',
 'knits',
 'outerwear',
 'lounge',
 'sweaters',
 'skirts',
 'fine gauge',
 'sleep',
 'jackets',
 'swim',
 'trend',
 'jeans',
 'legwear',
 'shorts',
 'layering']

In [161]:
# remove the stop words
def remove_stop(data):
    data=[word.lower()for word in data]
    return [word for word in data if word not in new_stopping_words]

In [162]:
# remove the number in review
def remove_num(data):
    return [word for word in data if word.isalpha()]

In [163]:
#lemmatization 
from nltk.stem import WordNetLemmatizer
def lemmatizze(data): 
    newdata= [WordNetLemmatizer().lemmatize(t) for t in data]
    return newdata

In [164]:
def clean_review(data):
    data1=split_method(data)
    data2=remove_stop(data1)
    data3=remove_num(data2)
    final_data=lemmatizze(data3)
    return ' '.join(final_data)

In [165]:
import nltk
nltk.download('wordnet')
WC['review_clean']=WC['Review Text'].apply(clean_review)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zheng\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [166]:
WC[['Review Text','review_clean']].head(5)

,Review Text,review_clean
0,Absolutely wonderful - silky and sexy and comf...,absolutely wonderful silky sexy comfortable
1,Love this dress! it is sooo pretty. i happen...,love dress sooo pretty happened find store gla...
2,I had such high hopes for this dress and reall...,high hope dress really wanted work initially o...
3,"I love, love, love this jumpsuit. it is fun, f...",love love love jumpsuit fun flirty fabulous ev...
4,This shirt is very flattering to all due to th...,shirt flattering due adjustable front tie perf...


# creating new features according to the review contents.

In [167]:
WC['length_review']=WC['review_clean'].apply(lambda x : len(x.split(' ')))

In [168]:
def number(data):
    return ' '.join(re.findall('([0-9]+\W?[0-9]+)',data))
def number_lbs(data):
    return ' '.join(re.findall('([0-9]+lb+.?)',data))


In [169]:
WC['number']=WC['Review Text'].apply(number)
WC['number_lbs']= WC['Review Text'].apply(number_lbs)

In [170]:
WC['have titile']=WC['Title'].notnull()

In [171]:
WC['have_number']= WC['number']!=''

In [172]:
WC['have_number']=WC['have_number']+0

In [173]:
WC['have titile']=WC['have titile']+0

In [174]:
WC.head(5)

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,review_clean,length_review,number,number_lbs,have titile,have_number
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,0,0,Initmates,Intimate,Intimates,absolutely wonderful silky sexy comfortable,5,,,0,0
1,1080,34,NaN,Love this dress! it is sooo pretty. i happen...,5,0,4,General,Dresses,Dresses,love dress sooo pretty happened find store gla...,27,5'8,,0,1
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,1,0,General,Dresses,Dresses,high hope dress really wanted work initially o...,50,,,1,0
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it is fun, f...",5,0,0,General Petite,Bottoms,Pants,love love love jumpsuit fun flirty fabulous ev...,15,,,1,0
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,0,6,General,Tops,Blouses,shirt flattering due adjustable front tie perf...,16,,,1,0


# 3.Building ML mode for recommendation prediction. 

# undersampling

In [175]:
# adjust the imbalance in train dataset. 
from imblearn.under_sampling import RandomUnderSampler
def adjust_balance(Xtrain,ytrain):
    rus = RandomUnderSampler(replacement=False)
    X_train_subsample, y_train_subsample = rus.fit_resample(Xtrain, ytrain)
    return X_train_subsample, y_train_subsample

# vectorization--TF-IDF

In [176]:
# review --- vector 
from sklearn.feature_extraction.text import TfidfVectorizer 
def review_to_vector(df_train,df_test):  # 产生df中不同的词的向量表示，一个大型的稀疏矩阵
    tf_idf_vectorizer = TfidfVectorizer(max_df=0.95,min_df=0.001)# min_df=0.001 
    tf_idf_vectorizer.fit(df_train)
    word_names= tf_idf_vectorizer.get_feature_names() 
    X_train_tf = tf_idf_vectorizer.transform(df_train).toarray()
    X_test_tf = tf_idf_vectorizer.transform(df_test).toarray()
    return X_train_tf,X_test_tf,word_names

# 3.4 Evaluate different models' performance. 

In [177]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [178]:
#同时报告这些参数
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import  f1_score
from sklearn.metrics import roc_curve, auc

# generate the train and test data

In [179]:
WC_analysis=WC.copy()
WC_analysis=pd.get_dummies(data=WC_analysis, columns=['Department Name'])

In [180]:
from sklearn.preprocessing import MinMaxScaler
std = MinMaxScaler()
WC_analysis[['Age','length_review']] = std.fit_transform(WC_analysis[['Age','length_review']])

In [181]:
WC_analysis.columns

Index(['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Class Name', 'review_clean', 'length_review', 'number', 'number_lbs',
       'have titile', 'have_number', 'Department Name_Bottoms',
       'Department Name_Dresses', 'Department Name_Intimate',
       'Department Name_Jackets', 'Department Name_Tops',
       'Department Name_Trend'],
      dtype='object')

In [182]:
# the new feature selected by us. 
X_list=['Age','length_review','Department Name_Bottoms','Department Name_Dresses', 'Department Name_Intimate',
       'Department Name_Jackets', 'Department Name_Tops','Department Name_Trend','review_clean']

In [183]:
X=WC_analysis[X_list]
y=WC_analysis[['Recommended IND']]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)
X_train_b,y_train_b=adjust_balance(X_train,y_train)

X_train_idf,X_test_idf,word_names=review_to_vector(X_train_b['review_clean'],X_test['review_clean'])
y_train=y_train_b['Recommended IND']
y_test=y_test['Recommended IND']

In [184]:
X_train_idf.shape

(5818, 2038)

# train models

In [36]:
def classification_task(m,X_test_scaled ,y_test,mn): 
    predic=m.predict(X_test_scaled)
    predict_probs=m.predict_proba(X_test_scaled)[:,1]
    fpr,tpr, thr=roc_curve(y_test,predict_probs)
    roc_auc=auc(fpr,tpr) 
    perf_df=pd.DataFrame({"Precision_Score":precision_score(y_test,predic),"Recall_Score":recall_score(y_test,predic),
                       "F1_Score":f1_score(y_test,predic) , "Accuracy":accuracy_score(y_test,predic),"AUC":roc_auc},index=[mn])
    return perf_df

In [185]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=3,max_features='auto',random_state=101)
clf.fit(X_train_idf, y_train)
feature_importance_=pd.DataFrame(clf.feature_importances_,index=word_names,columns=['feature_importance']).sort_values('feature_importance',ascending=False)
feature_importance_20=feature_importance_.head(20)
per=classification_task(clf,X_test_idf ,y_test,'GBDT')

In [106]:
feature_importance_.head(50)

,feature_importance
love,0.059930
perfect,0.059524
wanted,0.053419
great,0.049622
disappointed,0.046707
back,0.043305
looked,0.039993
comfortable,0.037969
not,0.031330
return,0.027376


In [56]:
feature_importance_.head(50).to_excel('feature_importance.xlsx')

In [39]:
word_names

['able',
 'absolutely',
 'accent',
 'accentuate',
 'accentuated',
 'accentuates',
 'acceptable',
 'accessory',
 'accidentally',
 'accommodate',
 'according',
 'accurate',
 'accurately',
 'across',
 'acrylic',
 'actual',
 'actually',
 'add',
 'added',
 'adding',
 'addition',
 'additional',
 'additionally',
 'adjust',
 'adjustable',
 'admit',
 'adn',
 'adorable',
 'adore',
 'advertised',
 'advice',
 'aesthetic',
 'afraid',
 'ag',
 'age',
 'ago',
 'agree',
 'ahead',
 'air',
 'airy',
 'ala',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'alteration',
 'altered',
 'alternative',
 'although',
 'always',
 'amazing',
 'amazingly',
 'amount',
 'amp',
 'ample',
 'ankle',
 'annoying',
 'another',
 'anticipated',
 'anymore',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'apart',
 'appeal',
 'appealing',
 'appear',
 'appearance',
 'appeared',
 'appears',
 'appreciate',
 'appropriate',
 'aqua',
 'area',
 'arenot',
 'arm',
 'armhole',
 'armpit',
 'around',
 'arrival',
 

In [186]:
NB=MultinomialNB()
NB.fit(X_train_idf, y_train)
perm_NB=classification_task(NB,X_test_idf ,y_test,'NB')
lr=LogisticRegression()
lr.fit(X_train_idf, y_train)
perm_LR=classification_task(lr,X_test_idf ,y_test,'LR')
rfc=RandomForestClassifier(n_estimators=100,random_state=101)
rfc.fit(X_train_idf, y_train)
perm_RFC=classification_task(rfc,X_test_idf ,y_test,'RFC')
per_model=pd.concat([per,perm_NB,perm_RFC,perm_LR])

In [194]:
per_model.to_excel('after_stop_words.xlsx')

# add new variables. --age,lenght_review and the department 

In [111]:
# add new feature
X_train_add=[]
X_test_add=[]
for i in range(X_train_idf.shape[0]):
    X_train_add.append(np.append(X_train_idf[i],X_train_b.iloc[i,:-1]))
for i in range(X_test_idf.shape[0]):
    X_test_add.append(np.append(X_test_idf[i],X_test.iloc[i,:-1]))

In [112]:
X_train_idf=np.array(X_train_add)
X_test_idf=np.array(X_test_add)
X_train_idf.shape

(5818, 2067)

In [113]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=3,max_features='auto',random_state=101)
clf.fit(X_train_idf, y_train)
per_GBDT_add=classification_task(clf,X_test_idf ,y_test,'GBDT_add')
NB=MultinomialNB()
NB.fit(X_train_idf, y_train)
perm_NB_add=classification_task(NB,X_test_idf ,y_test,'NB_add')
lr=LogisticRegression()
lr.fit(X_train_idf, y_train)
perm_LR_add=classification_task(lr,X_test_idf ,y_test,'LR_add')
rfc=RandomForestClassifier(n_estimators=100,random_state=101)
rfc.fit(X_train_idf, y_train)
perm_RFC_add=classification_task(rfc,X_test_idf ,y_test,'RFC_add')
per_model_add=pd.concat([per_GBDT_add,perm_NB_add,perm_RFC_add,perm_LR_add])

In [114]:
per_model_add

,Precision_Score,Recall_Score,F1_Score,Accuracy,AUC
GBDT_add,0.458119,0.821309,0.588165,0.798145,0.883796
NB_add,0.509425,0.861577,0.640274,0.830094,0.917994
RFC_add,0.500993,0.846477,0.629445,0.825088,0.908377
LR_add,0.531522,0.869966,0.659879,0.842609,0.922203


# imporve the performance of model by high-parameter tuning. 

In [58]:
from sklearn.model_selection import RandomizedSearchCV

In [72]:
params = {
        'n_estimators': [50,100, 150, 200],
        'learning_rate': [0.05, 0.10, 0.20],
        'max_depth': [3,4,5],
        'min_samples_split':[20,30,40,50]
        }

In [64]:
gbdt_class=GradientBoostingClassifier(max_features='auto',random_state=101)

random_search = RandomizedSearchCV(gbdt_class,param_distributions=params, n_iter=50, scoring='recall', n_jobs=-1, cv=5, verbose=3, random_state=1001 )

random_search.fit(X_train_idf, y_train)
print('\n Best estimator:')
print(random_search.best_estimator_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5,
                   estimator=GradientBoostingClassifier(max_features='auto',
                                                        random_state=101),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'learning_rate': [0.05, 0.1, 0.2],
                                        'max_depth': [3, 4, 5],
                                        'min_samples_split': [20, 30, 40, 50],
                                        'n_estimators': [50, 100, 150, 200]},
                   random_state=1001, scoring='recall', verbose=3)

In [190]:
clf_best = GradientBoostingClassifier(n_estimators=200,max_depth=5,min_samples_split=50,learning_rate=0.1,max_features='auto',random_state=101)
clf_best.fit(X_train_idf, y_train)
per_best=classification_task(clf_best,X_test_idf ,y_test,'GBDT_best')

In [191]:
per_best

,Precision_Score,Recall_Score,F1_Score,Accuracy,AUC
GBDT_best,0.50603,0.844799,0.632935,0.828033,0.907551


In [59]:
params_rfc = {
        'n_estimators': [100, 130,160,190],
        'max_depth': [3,5,7],
        'min_samples_split':[30,40,50]
        }
RFC_tune=RandomForestClassifier(random_state=101)
random_search = RandomizedSearchCV(RFC_tune,param_distributions=params_rfc, n_iter=50, scoring='recall', n_jobs=-1, cv=5, verbose=3, random_state=1001 )
random_search.fit(X_train_idf, y_train)
print('\n Best estimator:')
print(random_search.best_estimator_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits

 Best estimator:
RandomForestClassifier(max_depth=7, min_samples_split=50, n_estimators=190,
                       random_state=101)


In [192]:
rfc_best=RandomForestClassifier(n_estimators=160,random_state=101)
rfc_best.fit(X_train_idf, y_train)
perm_rfc_best=classification_task(rfc_best,X_test_idf ,y_test,'RFC_best')

In [193]:
perm_rfc_best

,Precision_Score,Recall_Score,F1_Score,Accuracy,AUC
RFC_best,0.501957,0.860738,0.634116,0.825677,0.912709
